# Importing required packages

In [18]:
from selenium import webdriver   
import requests,os
from selenium.webdriver.chrome.options import Options
import time     
from bs4 import BeautifulSoup
import lxml
import urllib
import numpy as np

In [19]:
base_url='https://gaana.com'

# Opening gaana.com url and prepare to scrape

In [3]:
option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")

# Pass the argument 1 to allow and 2 to block
option.add_experimental_option("prefs", { "profile.default_content_setting_values.notifications": 1 })

browser = webdriver.Chrome(executable_path='./chromedriver',options=option) 
browser.get(base_url)
browser.implicitly_wait(15)
try:
    browser.find_element_by_class_name('popup-close').click()
    browser.find_element_by_class_name('popup-close').click()
except:
    browser.implicitly_wait(2)
    pass

# Songs and artists scrapping function of latests songs

In [11]:
def subsong(browser,url):
    base_url='https://gaana.com'
    if not url.find('https://')>=0:
        url=base_url+url
    print(url)
    browser.get(url)
    soup=BeautifulSoup(browser.page_source,'lxml')
    songs=soup.find_all('a',attrs={'class':'sng_c'})
    songs+=soup.find_all('a',attrs={'class':'sng_c '})
    songs+=soup.find_all('a',attrs={'class':'album-name block_dotes _dotes pjax'})
    songs+=soup.find_all('a',attrs={'class':'imghover'})
    songs=[song for song in songs if not song['href'].startswith('javascript:')]
    return songs
   

# Scroller for all playlists

In [6]:
current={}
while True:
    try:
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        browser.implicitly_wait(15)
        page=browser.page_source
        soup=BeautifulSoup(page,'lxml')
        links=soup.find_all('div',attrs={'class':'carousel'})
        current_tag=[l.find('h2').text.replace('View all','').strip() for l in links]
        pre=len(current)
        current={cat:link.find('a')['href'] for cat,link in zip(current_tag,links) }
        post=len(current)
        if pre==post:
            break
    except Exception as e:
        print(e)
        

# Scrapping playlists

In [7]:
page=browser.page_source
soup=BeautifulSoup(page,'lxml')
playlists=soup.find_all('a',attrs={'class':'imghover'})
len(playlists)

218

# Scrapping Trending songs and artists

In [56]:
links=[]
for url in playlists:
    links+=subsong(browser,url['href'])
links=list(set(links))

https://gaana.com/playlist/gaana-dj-trending-songs-hindi
https://gaana.com/playlist/gaana-dj-new-release-hot-20-punjabi
https://gaana.com/playlist/gaana-dj-international-weekly-hot-20
https://gaana.com/season/90s-ki-cassette-with-manjula
https://gaana.com/playlist/gaana-dj-just-in-punjabi
https://gaana.com/playlist/gaana-dj-ode-to-a-sunny-day
https://gaana.com/playlist/gaana-dj-love-wali-feeling
https://gaana.com/playlist/gaana-dj-trending-around-you
https://gaana.com/playlist/gaana-dj-hindi-top-50-1
https://gaana.com/playlist/gaana-dj-punjabi-top-50-1
https://gaana.com/playlist/gaana-dj-gaana-international-top-50
https://gaana.com/playlist/gaana-dj-trending-songs-hindi
https://gaana.com/playlist/gaana-dj-new-release-hot-20-punjabi
https://gaana.com/playlist/gaana-dj-international-weekly-hot-20
https://gaana.com/playlist/gaana-dj-new-releases-hot-20-hindi
https://gaana.com/playlist/gaana-dj-todays-top-5-punjabi
https://gaana.com/playlist/gaana-dj-todays-top-5-international
https://gaan

https://gaana.com/visualstories/web-stories/best-sidhu-moose-wala-songs-of-all-times/amp_web_stories/77969422.cms
https://gaana.com/visualstories/web-stories/sweat-it-out-with-these-punjabi-songs/amp_web_stories/77848569.cms
https://gaana.com/visualstories/web-stories/top-amrit-maan-songs-for-your-playlist/amp_web_stories/77844814.cms
https://gaana.com/song/main-shair-to-nahin-2
https://gaana.com/song/backbone-48
https://gaana.com/song/time-in-a-bottle-8
https://gaana.com/song/dil-to-pagal-hai
https://gaana.com/song/high-rated-gabru
https://gaana.com/song/i-want-your-sex-3
https://gaana.com/song/tujhe-dekha-to
https://gaana.com/song/main-tenu-1
https://gaana.com/song/lets-stay-together-4
https://gaana.com/song/aankhon-ki-gustakhiyan-maaf-ho
https://gaana.com/playlist/gaana-dj-krishna
https://gaana.com/playlist/gaana-dj-guru-nanak-dev
https://gaana.com/playlist/gaana-dj-Popular-collections-of-shiva
https://gaana.com/playlist/gaana-dj-guru-angad-dev
https://gaana.com/playlist/gaana-dj-Po

# Filtering Song and artists from links of playlists

In [57]:
csongs=[]
artists=[]
for l in links:
    if l['href'].startswith('/artist'):
        artists.append(l)
    else:csongs.append(l)
print(len(csongs))
print(len(artists))

3146
3169


In [64]:
x=[]
for s in csongs:
    x.append(s.text+'  '+s['href'])

# Scrapping all songs of each artist 

In [50]:
def artist_songs_f(browser,artist_link):
    if not artist_link.startswith('https'):
        artist_link=base_url+artist_link
    browser.get(artist_link)
    browser.implicitly_wait(2)
    try:
        soup=BeautifulSoup(browser.page_source,'lxml')
        records=int(soup.find('p',attrs={'id':'artist-record'}).text.replace('Tracks','').strip())
    except Exception as e:
        records=1
    for _ in range(records//40):
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        browser.implicitly_wait(3)
    soup=BeautifulSoup(browser.page_source,'lxml')
    songs=[song.find('a',attrs={'class':'sng_c'}) for song in soup.find_all('div',attrs={'class':'playlist_thumb_det'})]
    return songs

artist_songs=dict()
for artist_a in artists:
    print(artist_a['href'])
    artist_songs[artist_a['href'][8:]]=artist_songs_f(browser,artist_a['href'])

In [ ]:
artist_songs